# Obtención datos

In [2]:
from statsbombpy import sb
import pandas as pd
import numpy as np
import json
import os
import pymongo
from matplotlib.colors import LinearSegmentedColormap
import streamlit as st
import pandas as pd 
from data import data
import json
import numpy as np
from streamlit_option_menu import option_menu
from mplsoccer import VerticalPitch,Pitch
from highlight_text import ax_text, fig_text
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
from IPython.display import Image

pd.options.display.max_columns = None
pd.options.display.max_rows = 10

import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [2]:
#Tabla de las competiciones abiertas de StatsBomb
free_comps = sb.competitions()
free_comps

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-12-12T07:43:33.436182,None,None,2023-12-12T07:43:33.436182
1,16,4,Europe,Champions League,male,False,False,2018/2019,2023-03-07T12:20:48.118250,2021-06-13T16:17:31.694,None,2023-03-07T12:20:48.118250
2,16,1,Europe,Champions League,male,False,False,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-01-23T21:55:30.425330
3,16,2,Europe,Champions League,male,False,False,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
4,16,27,Europe,Champions League,male,False,False,2015/2016,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
...,...,...,...,...,...,...,...,...,...,...,...,...
65,55,43,Europe,UEFA Euro,male,False,True,2020,2023-02-24T21:26:47.128979,2023-04-27T22:38:34.970148,2023-04-27T22:38:34.970148,2023-02-24T21:26:47.128979
66,35,75,Europe,UEFA Europa League,male,False,False,1988/1989,2023-06-18T19:28:39.443883,2021-06-13T16:17:31.694,None,2023-06-18T19:28:39.443883
67,53,106,Europe,UEFA Women's Euro,female,False,True,2022,2023-10-24T03:36:54.066267,2023-10-24T03:37:29.085948,2023-10-24T03:37:29.085948,2023-10-24T03:36:54.066267
68,72,107,International,Women's World Cup,female,False,True,2023,2023-12-12T14:06:50.626363,2023-12-12T14:12:41.561162,2023-12-12T14:12:41.561162,2023-12-12T14:06:50.626363


In [3]:
bundesliga = partidos = sb.matches(competition_id=11, season_id=27)
bundesliga.home_team.unique()

array(['Levante UD', 'Las Palmas', 'RC Deportivo La Coruña', 'Málaga',
       'Espanyol', 'Sporting Gijón', 'Rayo Vallecano', 'Real Betis',
       'Athletic Club', 'Atlético Madrid', 'Valencia', 'Eibar', 'Getafe',
       'Villarreal', 'Sevilla', 'Granada', 'Real Sociedad', 'Celta Vigo',
       'Real Madrid', 'Barcelona'], dtype=object)

Se obtienen los datos correspondientes a las siguientes competiciones:

    - LaLiga 2015/16: competition_id:11 y season_id:27.
    - Serie A 2015/16: competition_id:12 y season_id:27.
    - Premier League 2015/16: competition_id:2 y season_id:27.
    - Bundesliga 2015/16: competition_id:9 y season_id: 27.

Esta función se incluye después en data.py para poder utilizar cada una de las jornadas de cada una de las ligas, con la jornada, el nombre de los equipos y el resultado. Estos valores están asociados al identificador de cada uno de los partidos, para poder mockearlos.

In [4]:

def obtencionPartidosJornada(competition_id, season_id):
    
    partidos = sb.matches(competition_id=competition_id, season_id=season_id)
    partidos = partidos.sort_values('match_week')

    jornadas={}

    for index, partido in partidos.iterrows():
        nombre_partido = f'Jornada {partido.match_week}: {partido.home_team} {partido.home_score} - {partido.away_score} {partido.away_team}'
        identificador_partido = partido.match_id

        jornadas.update({nombre_partido: identificador_partido})

    return jornadas

obtencionPartidosJornada(11, 27)

{'Jornada 1: Rayo Vallecano 0 - 0 Valencia': 3825566,
 'Jornada 1: Sporting Gijón 0 - 0 Real Madrid': 3825567,
 'Jornada 1: Real Betis 1 - 1 Villarreal': 3825568,
 'Jornada 1: Levante UD 1 - 2 Celta Vigo': 3825569,
 'Jornada 1: Málaga 0 - 0 Sevilla': 3825562,
 'Jornada 1: Atlético Madrid 1 - 0 Las Palmas': 3825563,
 'Jornada 1: RC Deportivo La Coruña 0 - 0 Real Sociedad': 3825564,
 'Jornada 1: Espanyol 1 - 0 Getafe': 3825565,
 'Jornada 1: Athletic Club 0 - 1 Barcelona': 266236,
 'Jornada 1: Granada 1 - 3 Eibar': 3825570,
 'Jornada 2: Eibar 2 - 0 Athletic Club': 3825578,
 'Jornada 2: Valencia 1 - 1 RC Deportivo La Coruña': 3825576,
 'Jornada 2: Villarreal 3 - 1 Espanyol': 3825571,
 'Jornada 2: Real Sociedad 0 - 0 Sporting Gijón': 3825573,
 'Jornada 2: Real Madrid 5 - 0 Real Betis': 3825572,
 'Jornada 2: Sevilla 0 - 3 Atlético Madrid': 3825579,
 'Jornada 2: Las Palmas 0 - 0 Levante UD': 3825575,
 'Jornada 2: Celta Vigo 3 - 0 Rayo Vallecano': 3825574,
 'Jornada 2: Getafe 1 - 2 Granada': 3

Función para la obtención de todos los partidos de la temporada 2015/16 con sus correspondientes eventos.

In [5]:
def descargarEventosTemporada(id_competicion, identificador_temporada, directorio_destino):

    # Obtener los identificadores de los partidos de la temporada
    partidos_temporada = sb.matches(competition_id=id_competicion, season_id=identificador_temporada)
    identificadores_partidos = partidos_temporada.match_id.unique()

    # Crear el directorio de destino si no existe
    if not os.path.exists(directorio_destino):
        os.makedirs(directorio_destino)

    # Descargar los eventos de cada partido
    for partido_id in identificadores_partidos:
        eventos_partido = sb.events(match_id=partido_id)
        eventos_partido = eventos_partido.to_json(orient='records')
        archivo_destino = os.path.join(directorio_destino, f'{partido_id}.json')

        try:
            with open(archivo_destino, 'w') as archivo:
                archivo.write(str(eventos_partido))
        except OSError as e:
            print(f'Error al guardar los eventos del partido {partido_id}:', e)
            continue

        print(f'Se ha descargado el archivo JSON del partido {partido_id}.')

    print('¡Se han descargado todos los archivos JSON de eventos de la temporada!')


Descarga de los partidos de cada una de las competiciones y se guardan dentro de una carpeta especifica de games_json/

In [6]:
descargarEventosTemporada(id_competicion=11, identificador_temporada=27, directorio_destino='games_json/LaLiga15')
descargarEventosTemporada(id_competicion=12, identificador_temporada=27, directorio_destino='games_json/SerieA15')
descargarEventosTemporada(id_competicion=2, identificador_temporada=27, directorio_destino='games_json/Premier15')
descargarEventosTemporada(id_competicion=9, identificador_temporada=27, directorio_destino='games_json/Bundesliga15')

Se ha descargado el archivo JSON del partido 3825848.
Se ha descargado el archivo JSON del partido 3825895.
Se ha descargado el archivo JSON del partido 3825894.
Se ha descargado el archivo JSON del partido 3825855.
Se ha descargado el archivo JSON del partido 3825908.
Se ha descargado el archivo JSON del partido 3825883.
Se ha descargado el archivo JSON del partido 3825900.
Se ha descargado el archivo JSON del partido 3825902.
Se ha descargado el archivo JSON del partido 3825876.
Se ha descargado el archivo JSON del partido 3825846.
Se ha descargado el archivo JSON del partido 3825869.
Se ha descargado el archivo JSON del partido 3825882.
Se ha descargado el archivo JSON del partido 3825849.
Se ha descargado el archivo JSON del partido 3825860.
Se ha descargado el archivo JSON del partido 3825847.
Se ha descargado el archivo JSON del partido 3825817.
Se ha descargado el archivo JSON del partido 3825778.
Se ha descargado el archivo JSON del partido 3825777.
Se ha descargado el archivo 

Ejemplo de código para obtener los datos de fbref